In [1]:
!pip install pyspark
!pip install findspark

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 783.7 kB/s eta 0:00:00 0:00:01m
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.5
    Uninstalling py4j-0.10.9.5:
      Successfully uninstalled py4j-0.10.9.5


In [2]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [5]:
if SparkContext._active_spark_context:
    SparkContext._active_spark_context.stop()

sc = SparkContext()

In [6]:
SPARK_MASTER_URL = "spark://spark-master:7077"

spark = SparkSession.builder \
    .master(SPARK_MASTER_URL) \
    .appName("Jupyter-Spark-K8s") \
    .config("spark.driver.memory", "2g") \
    .getOrCreate()

print("✅ Conectado a Spark en Kubernetes")

✅ Conectado a Spark en Kubernetes


In [11]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv

--2025-02-14 00:09:15--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/searchterms.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 233457 (228K) [text/csv]
Saving to: ‘searchterms.csv.2’

searchterms.csv.2   100%[===================>] 227.99K   755KB/s    in 0.3s    

2025-02-14 00:09:16 (755 KB/s) - ‘searchterms.csv.2’ saved [233457/233457]



In [15]:
df = spark.read.csv("searchterms.csv", header=True, inferSchema=True)
df.show(5)

+---+-----+----+--------------+
|day|month|year|    searchterm|
+---+-----+----+--------------+
| 12|   11|2021| mobile 6 inch|
| 12|   11|2021| mobile latest|
| 12|   11|2021|   tablet wifi|
| 12|   11|2021|laptop 14 inch|
| 12|   11|2021|     mobile 5g|
+---+-----+----+--------------+
only showing top 5 rows



In [20]:
df.select("searchterm").printSchema()

root
 |-- searchterm: string (nullable = true)



In [18]:
df.count()

10000

In [19]:
len(df.columns)

4

In [23]:
from pyspark.sql.functions import col,desc
df.filter(col("searchterm") == "gaming laptop").count()

499

In [25]:
df.groupBy("searchterm").count().orderBy(desc("count")).limit(5).show()

+-------------+-----+
|   searchterm|count|
+-------------+-----+
|mobile 6 inch| 2312|
|    mobile 5g| 2301|
|mobile latest| 1327|
|       laptop|  935|
|  tablet wifi|  896|
+-------------+-----+



In [26]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz

--2025-02-14 00:28:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DB0321EN-SkillsNetwork/Bigdata%20and%20Spark/model.tar.gz
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1490 (1.5K) [application/x-tar]
Saving to: ‘model.tar.gz’

model.tar.gz        100%[===================>]   1.46K  --.-KB/s    in 0s      

2025-02-14 00:28:44 (1.01 GB/s) - ‘model.tar.gz’ saved [1490/1490]



In [28]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
import tarfile

In [32]:
from pyspark.ml.regression import LinearRegressionModel

In [30]:
!tar -xvzf model.tar.gz

sales_prediction.model/
sales_prediction.model/metadata/
sales_prediction.model/metadata/part-00000
sales_prediction.model/metadata/.part-00000.crc
sales_prediction.model/metadata/_SUCCESS
sales_prediction.model/metadata/._SUCCESS.crc
sales_prediction.model/data/
sales_prediction.model/data/part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet
sales_prediction.model/data/_SUCCESS
sales_prediction.model/data/.part-00000-1db9fe2f-4d93-4b1f-966b-3b09e72d664e-c000.snappy.parquet.crc
sales_prediction.model/data/._SUCCESS.crc


In [34]:
model = LinearRegressionModel.load("sales_prediction.model")

In [41]:
from pyspark.ml.feature import VectorAssembler

from pyspark.ml.feature import VectorAssembler

def predict(year):
    assembler = VectorAssembler(inputCols=["year"], outputCol="features")
    data = [[year, 0]]
    columns = ["year", "sales"]
    df = spark.createDataFrame(data, columns)
    transformed_df = assembler.transform(df).select('features', 'sales')
    predictions = model.transform(transformed_df)
    predictions.select('prediction').show()


In [42]:
predict(2023)

+------------------+
|        prediction|
+------------------+
|175.16564294006457|
+------------------+

